## 2 Multiprocessing in Python

Unix and Linux operating systems
* create child processes : `fork()`   The parent process that calls the `fork()` function to creates a child process, which is a copy of the parent process but has its own PID. 

Windows OS:
* create child processes : use the `Process` class from the `multiprocessing` module.   
* `multiprocessing` module :
	* offer process pools (`Pool`) for batch process launching, 
	* queues (`Queue`) and pipes (`Pipe`) for inter-process communication


In [3]:
## see script\s008_sample_multiprocessing1.py

Total used of 0.13 seconds.


## 2 Multiprocessing in Python

### 2.1 threading

Python in early versions, the `thread` module (now named `_thread`) was introduced for multithreading programming. However, this module is quite low-level and lacks many features.

For Python multithreading development, it is recommended to use the `threading` module, which provides a better object-oriented encapsulation for multithreaded programming.

In [7]:
"""
Use `Thread` class in threading module to create thread
the usage is similar to Process Class
    Parameter:
        target    : exec function name
        arg       : arg to function
    Functions:
        thread.start()           : start thread
        thread.join([timeout])   : wait thread finish
    
"""
from random import randint
from threading import Thread
from time import time, sleep


def download(filename):
    print('Start download %s...' % filename)
    time_to_download = randint(5, 10)
    sleep(time_to_download)
    print('%s Complete! Total time spent:  %d seconds' % (filename, time_to_download))

def main():
    start = time()
    t1 = Thread(target=download, args=('Python从入门到住院.pdf',))
    t1.start()
    t2 = Thread(target=download, args=('Peking Hot.avi',))
    t2.start()
    t1.join()
    t2.join()
    end = time()
    print('Total time spent:  %.2f seconds.' % (end - start))

if __name__ == '__main__':
    main()

Start download Python从入门到住院.pdf...
Start download Peking Hot.avi...
Python从入门到住院.pdf Complete! Total time spent:  6 seconds
Peking Hot.avi Complete! Total time spent:  8 seconds
Total time spent:  8.02 seconds.


In [6]:
"""
Create a custome thread by Inheriting from Thread Class, and then create thread objects and start the threads.
"""
from random import randint
from threading import Thread
from time import time, sleep


class DownloadTask(Thread):

    def __init__(self, filename):
        super().__init__()
        self._filename = filename

    def run(self):
        print('Start Download %s...' % self._filename)
        time_to_download = randint(5, 10)
        sleep(time_to_download)
        print('%sDownload Complete! Speed %d Seconds' % (self._filename, time_to_download))


def main():
    start = time()
    t1 = DownloadTask('Python从入门到住院.pdf')
    t1.start()
    t2 = DownloadTask('Peking Hot.avi')
    t2.start()
    t1.join()
    t2.join()
    end = time()
    print('Total time spent: %.2f seconds.' % (end - start))


if __name__ == '__main__':
    main()

Start Download Python从入门到住院.pdf...
Start Download Peking Hot.avi...
Python从入门到住院.pdfDownload Complete! Speed 8 Seconds
Peking Hot.aviDownload Complete! Speed 8 Seconds
Total time spent: 8.01 seconds.


### 2.2 Lock

  
When multiple threads share the same resource (variable), it can lead to unpredictable(不可控) results, potentially causing the program to fail or crash. In such cases, a "`lock`" can be useful.
We can use locks to protect the same resource. **Only the thread that acquires the lock(获得锁) can access this resource**, while other threads that do not have the lock will be blocked until the thread that holds the lock releases it. 


In [8]:
from time import sleep
from threading import Thread, Lock

class Account(object):

    def __init__(self):
        self._balance = 0
        self._lock = Lock()

    def deposit(self, money):
        # Only acquires the lock can run the below code
        self._lock.acquire()
        try:
            new_balance = self._balance + money
            sleep(0.01)
            self._balance = new_balance
        finally:
            # Ensure that locks are properly released
            self._lock.release()

    @property
    def balance(self):
        return self._balance


class AddMoneyThread(Thread):

    def __init__(self, account, money):
        super().__init__()
        self._account = account
        self._money = money

    def run(self):
        self._account.deposit(self._money)


def main():
    account = Account()
    threads = []
    for _ in range(100):
        t = AddMoneyThread(account, 1)            # init thread 
        threads.append(t)                         # collect all threads
        t.start()                                 # run thread
    for t in threads:
        t.join()                                  # wait thread finish
    print('Accout Balance: ￥%d Yuan' % account.balance)


if __name__ == '__main__':
    main()

Accout Balance: ￥100 Yuan
